# Requirement
- Get Total Quantity and Total Sales by week, by store, by Department, Category etc.

# Loading big data
- The data is not available in git repo as it is too big
- Download it before the class

In [1]:
import os
import pandas as pd

df_products = pd.read_parquet('./data/PRODUCTS/products.parquet')

In [2]:
df_stores = pd.read_parquet('./data/STORES/stores.parquet')

In [3]:
df_transactions = pd.read_parquet('./data/TRANSACTIONS/part-000000')

In [4]:
df_transactions["TransactionDate"] = pd.to_datetime(df_transactions['TransactionDate'])

# Below code on a 16gb, 4 core cpu mac takes more than 5 min

In [ ]:
%%time
df_merge_tran_product = pd.merge(df_transactions, df_products, how='left', left_on=['UPC'], right_on = ['UPC'])
del df_products

In [ ]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
df_merge_tran_product.groupby(['CATEGORY','week_number_of_year'])['Qty','SoldRate'].sum().rename(columns={'Qty':'Total Qty','SoldRate' : 'Total Sales'})

# Do with Spark

In [ ]:
# The Python packaging for Spark is not intended to replace all of the other use cases. 
# This Python packaged version of Spark is suitable for interacting with an existing cluster (be it Spark standalone, YARN, or Mesos) - but does not contain the tools required to setup your own standalone Spark cluster. 
# You can download the full version of Spark from the Apache Spark downloads page.
!pip3 install pyspark==3.0.0

In [ ]:
# Below variables are to be set in the shell profile
# export SPARK_HOME=/Users/pmacharl/spark-2.4.4-bin-hadoop2.7
# export PATH=$PATH:$SPARK_HOME/bin
# export PYSPARK_SUBMIT_ARGS="pyspark-shell"
# export PYSPARK_DRIVER_PYTHON=/usr/local/bin/python3
# export PYSPARK_PYTHON=/usr/local/bin/python3

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkConf
config = SparkConf()
config.set("spark.driver.memory", "2g")
config.set("spark.executor.memory", "1g")

#Because you are likely running in local mode, it is a good practice to set the number of shuffle partitions
# to something that is going to fit local mode. By default, the value is 200, but there aren't many executors
# on this machine, its worth reducing this to 5
config.set("spark.sql.shuffle.partitions", "5")

# Cluster mode
# https://spark.apache.org/docs/latest/submitting-applications.html
# config.setMaster("spark://192.168.0.7:7077") # If spark is started in local cluster mode

In [2]:
spark = SparkSession.builder.config(conf=config).master("local").appName("MyApp").getOrCreate()
spark

# Using DataFrames

In [ ]:
df_products = spark.read.load("./data/PRODUCTS")
df_products.show()

In [4]:
df_products.createOrReplaceTempView("products")

In [ ]:
df_transactions = spark.read.load("./data/TRANSACTIONS")
df_transactions.show()

In [6]:
df_transactions.createOrReplaceTempView("transactions")

In [7]:
df_join_product_transactions = df_transactions.join(df_products, df_transactions.UPC == df_products.UPC, 'inner')

In [8]:
%%time
df_join_product_transactions.groupby("DEPARTMENT").agg({"SoldRate": "sum", "Qty": "count"}).show()

+--------------------+----------+--------------------+
|          DEPARTMENT|count(Qty)|       sum(SoldRate)|
+--------------------+----------+--------------------+
|              Liquor|  42612676| 4.681069539636936E8|
|            Pharmacy|    496880|   4040946.220001863|
|Deli and Foodservice|  22348130|1.0888377814712748E8|
|             Grocery| 565333021| 1.514952610701241E9|
|             Produce| 237301076| 5.579940860804292E8|
|                 HBC|  23985791|1.3116028896553218E8|
|             Seafood|   2209534|1.1060811070005694E7|
|       Packaged Deli|  37220184|1.2202639257678652E8|
|                  GM|  70948513|3.8092355814723516E8|
|                Meat|  14072935| 7.245627896076763E7|
|               Dairy| 119286406|3.3135169825598425E8|
|              Frozen|  85128987|3.0558685919974375E8|
|          major misc| 142043830| 1.049723286940231E9|
|              Bakery|   7236959|2.7038583390471153E7|
|              Floral|   1194587|   3493103.259998988|
+---------

# Using Spark SQL
- [Spark SQL JOINS](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-joins.html)

In [ ]:
df_join_tran_products = spark.sql("select p.DEPARTMENT as Department, count(t.Qty) as TotalQty, sum(t.SoldRate) as TotalSales from transactions t join products p on t.UPC=p.UPC GROUP BY p.DEPARTMENT")

In [ ]:
%%time
from pyspark.sql.types import *
df_join_tran_products.withColumn('TotalSales', df_join_tran_products.TotalSales.cast(DecimalType(18, 2))).show()

In [ ]:
spark.stop()

# Simulating more data 
- Use Dbeaver (version < 6.2 , as they made "Generate mock data" feature enterprise from 6.2)
- Generate mock data is intuitive to use (play with it for some time and it will be easier)
- Understanding the sequence of loading tables ensures PK-FK validations are automatically taken care of for e.g. generating store records with address_ids that don't exist in address table doesn't make sense (so select FK for address_id field)